In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer2 import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
locs += np.array([0,0,-0.70])
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7002
INFO:drake:Meshcat listening for connections at http://localhost:7003


In [5]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

In [7]:
visualizer.meshcat2.Delete("/prm")

In [8]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plotting_fn(nodes, adjacency_list, width):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = Rgba(0.0, 0.0, 1, 1))
            plt_idx +=1
            
plotting_fn_handle = partial(plotting_fn, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)

PRM = prm.PRM( 
            limits_t,
            num_points = 500,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

# PRM.add_start_end(start, target)
PRM.plot()
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Samples 120
[PRM] Samples 150
[PRM] Samples 180
[PRM] Samples 210
[PRM] Samples 240
[PRM] Samples 270
[PRM] Samples 300
[PRM] Samples 330
[PRM] Samples 360
[PRM] Samples 390
[PRM] Samples 420
[PRM] Samples 450
[PRM] Samples 480
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
[PRM] Nodes connected: 100
[PRM] Nodes connected: 120
[PRM] Nodes connected: 140
[PRM] Nodes connected: 160
[PRM] Nodes connected: 180
[PRM] Nodes connected: 200
[PRM] Nodes connected: 220
[PRM] Nodes connected: 240
[PRM] Nodes connected: 260
[PRM] Nodes connected: 280
[PRM] Nodes connected: 300
[PRM] Nodes connected: 320
[PRM] Nodes connected: 340
[PRM] Nodes connected: 360
[PRM] Nodes connected: 380
[PRM] Nodes connected: 400
[PRM] Nodes connected: 420
[PRM] Nodes connected: 440
[PRM] Nodes connected: 460
[PRM] Nodes connected: 480


## Set up the sliders so we can move the plant around manually

In [9]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [10]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


In [11]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, np.zeros(2),
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

Time to construct line certifier = 20.754008531570435s


In [12]:
PRM.nodes

array([[ 2.66721491e-01, -1.30358189e-01],
       [-1.72442732e-01, -9.08667552e-01],
       [-1.16187026e-01, -1.13248913e+00],
       [-1.87898439e-02,  2.67496917e-01],
       [ 1.86311156e-01, -1.06693738e+00],
       [ 4.69338367e-03, -1.44592157e-02],
       [ 3.27204705e-01,  7.81296892e-02],
       [-5.02852737e-01, -8.29931208e-01],
       [-1.23193583e-01, -8.65817237e-01],
       [-1.56652986e-01, -8.29557201e-01],
       [-1.57096598e-01, -8.24489973e-01],
       [-2.82888970e-01, -5.18864759e-01],
       [-4.80546168e-01, -9.15194838e-01],
       [-6.17425209e-01, -8.40438053e-01],
       [-5.26356621e-01, -2.48037946e-01],
       [-6.09611482e-01, -1.02152176e+00],
       [ 3.77563671e-01,  1.45903510e-01],
       [-2.16847139e-01, -8.15730483e-01],
       [ 4.88137071e-04,  3.91361705e-02],
       [ 7.07099346e-02, -4.28435883e-02],
       [ 3.15328367e-01, -2.46323898e-01],
       [-4.95547270e-01, -3.01725558e-01],
       [ 3.45957659e-01, -1.12008427e+00],
       [ 3.

In [14]:
s0 = PRM.nodes[PRM.adjacency_list[0][0]]
uncertified_edges = []
print(s0)
t0_tot = time.time()
for item_num, item in enumerate(PRM.adjacency_list):
    s0 = PRM.nodes[item[0]]
    if (item_num % 100 == 0):
        print(f"Doing {item_num}/{len(PRM.adjacency_list)}")
    for idx1 in item[1:]:
        s1 = PRM.nodes[idx1]
        t0 = time.time()
        safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
        t1 = time.time()
#         print(f"Edge ({item[0]}, {    for idx1 in item[1:]:
# }) is {safe}")
#         print(f"Certified in {t1-t0}s")
#         print()
        if not safe:
            uncertified_edges.append([item[0], idx1])
t1_tot = time.time()
print(f"Certification in {t1_tot-t0_tot}s")

[ 0.26672149 -0.13035819]
Doing 0/500
Doing 100/500
Doing 200/500
Doing 300/500
Doing 400/500
Certification in 1920.2652184963226s


In [15]:
print(len(uncertified_edges))

91


In [ ]:

line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kConstant, np.zeros(2),
                                set(), VerificationOption()
                               )
s0 = PRM.nodes[PRM.adjacency_list[0][0]]
uncertified_edges = []
print(s0)
t0_tot = time.time()
for item_num, item in enumerate(PRM.adjacency_list):
    s0 = PRM.nodes[item[0]]
    if (item_num % 100 == 0):
        print(f"Doing {item_num}/{len(PRM.adjacency_list)}")
    for idx1 in item[1:]:
        s1 = PRM.nodes[idx1]
        t0 = time.time()
        safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
        t1 = time.time()
        print(f"Edge ({item[0]}, {idx1}) is {safe}")
        print(f"Certified in {t1-t0}s")
        print()
        if not safe:
            uncertified_edges.append([item[0], idx1])
t1_tot = time.time()
print(f"Certification in {t1_tot-t0_tot}s")

[ 0.33113314 -0.68168117]
Doing 0/500
Edge (0, 227) is False
Certified in 0.5110836029052734s

Edge (0, 183) is False
Certified in 0.6018214225769043s

Edge (0, 399) is False
Certified in 0.5287368297576904s

Edge (0, 447) is False
Certified in 0.5278973579406738s

Edge (1, 14) is False
Certified in 0.4906632900238037s

Edge (1, 112) is False
Certified in 0.5742239952087402s

Edge (1, 333) is False
Certified in 0.5316805839538574s

Edge (1, 478) is False
Certified in 0.49370598793029785s

Edge (2, 168) is False
Certified in 0.5033512115478516s

Edge (2, 344) is False
Certified in 0.45416998863220215s

Edge (2, 79) is False
Certified in 0.49568819999694824s

Edge (2, 4) is False
Certified in 0.5317902565002441s

Edge (3, 77) is False
Certified in 0.5036113262176514s

Edge (3, 125) is False
Certified in 0.5167913436889648s

Edge (3, 461) is False
Certified in 0.46570611000061035s

Edge (3, 122) is False
Certified in 0.5501627922058105s

Edge (4, 131) is False
Certified in 0.4767396450042

Edge (35, 139) is False
Certified in 0.560624361038208s

Edge (35, 83) is False
Certified in 0.46958065032958984s

Edge (35, 481) is False
Certified in 0.4942891597747803s

Edge (36, 325) is False
Certified in 0.5258054733276367s

Edge (36, 393) is False
Certified in 0.5368330478668213s

Edge (36, 487) is False
Certified in 0.4870429039001465s

Edge (36, 190) is False
Certified in 0.5285067558288574s

Edge (37, 464) is False
Certified in 0.5164599418640137s

Edge (37, 253) is False
Certified in 0.5284645557403564s

Edge (37, 206) is False
Certified in 0.5179197788238525s

Edge (37, 234) is False
Certified in 0.5523402690887451s

Edge (38, 283) is False
Certified in 0.4927668571472168s

Edge (38, 304) is False
Certified in 0.5298404693603516s

Edge (38, 444) is False
Certified in 0.5413782596588135s

Edge (38, 116) is False
Certified in 0.5486516952514648s

Edge (39, 155) is False
Certified in 0.5033435821533203s

Edge (39, 279) is False
Certified in 0.5393455028533936s

Edge (39, 387) 

Edge (73, 59) is False
Certified in 0.4875202178955078s

Edge (73, 413) is False
Certified in 0.5598165988922119s

Edge (74, 81) is False
Certified in 0.48836469650268555s

Edge (74, 45) is False
Certified in 0.5104351043701172s

Edge (74, 337) is False
Certified in 0.48288965225219727s

Edge (74, 123) is False
Certified in 0.47837018966674805s

Edge (75, 18) is False
Certified in 0.5348923206329346s

Edge (75, 448) is False
Certified in 0.4911017417907715s

Edge (75, 58) is False
Certified in 0.5134074687957764s

Edge (75, 173) is False
Certified in 0.5838825702667236s

Edge (76, 171) is False
Certified in 0.5455303192138672s

Edge (76, 140) is False
Certified in 0.5417304039001465s

Edge (76, 328) is False
Certified in 0.5457780361175537s

Edge (76, 338) is False
Certified in 0.49944353103637695s

Edge (77, 125) is False
Certified in 0.4786860942840576s

Edge (77, 3) is False
Certified in 0.5726103782653809s

Edge (77, 461) is False
Certified in 0.5426080226898193s

Edge (77, 306) is

Edge (117, 375) is False
Certified in 0.5320439338684082s

Edge (117, 455) is False
Certified in 0.5631289482116699s

Edge (117, 52) is False
Certified in 0.5266950130462646s

Edge (117, 264) is False
Certified in 0.5039288997650146s

Edge (118, 370) is False
Certified in 0.5113015174865723s

Edge (118, 108) is False
Certified in 0.49330878257751465s

Edge (118, 412) is False
Certified in 0.533747673034668s

Edge (118, 217) is False
Certified in 0.5317723751068115s

Edge (119, 169) is False
Certified in 0.5029172897338867s

Edge (119, 113) is False
Certified in 0.48689723014831543s

Edge (119, 97) is False
Certified in 0.4885118007659912s

Edge (119, 313) is False
Certified in 0.5242159366607666s

Edge (120, 105) is False
Certified in 0.4815678596496582s

Edge (120, 178) is False
Certified in 0.5212233066558838s

Edge (120, 100) is False
Certified in 0.4729878902435303s

Edge (120, 230) is False
Certified in 0.531358003616333s

Edge (121, 137) is False
Certified in 0.542515754699707s



Edge (152, 247) is False
Certified in 0.4985208511352539s

Edge (152, 214) is False
Certified in 0.578254222869873s

Edge (152, 343) is False
Certified in 0.539741039276123s

Edge (152, 89) is False
Certified in 0.5272359848022461s

Edge (153, 203) is False
Certified in 0.558096170425415s

Edge (153, 264) is False
Certified in 0.46423935890197754s

Edge (153, 149) is False
Certified in 0.5356559753417969s

Edge (153, 335) is False
Certified in 0.5122125148773193s

Edge (154, 158) is False
Certified in 0.4862654209136963s

Edge (154, 94) is False
Certified in 0.5162613391876221s

Edge (154, 299) is False
Certified in 0.5299570560455322s

Edge (154, 124) is False
Certified in 0.5010302066802979s

Edge (155, 39) is False
Certified in 0.5127973556518555s

Edge (155, 279) is False
Certified in 0.5241832733154297s

Edge (155, 167) is False
Certified in 0.5395171642303467s

Edge (155, 441) is False
Certified in 0.5521626472473145s

Edge (156, 115) is False
Certified in 0.4781019687652588s

Ed

...

Edge (229, 466) is False
Certified in 0.4764699935913086s

Edge (230, 178) is False
Certified in 0.4543161392211914s

Edge (230, 105) is False
Certified in 0.46913576126098633s

Edge (230, 19) is False
Certified in 0.5360898971557617s

Edge (230, 212) is False
Certified in 0.47127223014831543s

Edge (231, 74) is False
Certified in 0.5019204616546631s

Edge (231, 123) is False
Certified in 0.5824806690216064s

Edge (231, 81) is False
Certified in 0.49735450744628906s

Edge (231, 200) is False
Certified in 0.5330941677093506s

Edge (232, 291) is False
Certified in 0.49943041801452637s

Edge (232, 300) is False
Certified in 0.5197508335113525s

Edge (232, 62) is False
Certified in 0.4825007915496826s

Edge (232, 411) is False
Certified in 0.49948620796203613s

Edge (233, 7) is False
Certified in 0.5406143665313721s

Edge (233, 438) is False
Certified in 0.5269734859466553s

Edge (233, 424) is False
Certified in 0.5581223964691162s

Edge (233, 482) is False
Certified in 0.568622350692749s


Edge (267, 212) is False
Certified in 0.522003173828125s

Edge (268, 71) is False
Certified in 0.6133542060852051s

Edge (268, 174) is False
Certified in 0.531834602355957s

Edge (268, 11) is False
Certified in 0.5457165241241455s

Edge (268, 161) is False
Certified in 0.46499013900756836s

Edge (269, 179) is False
Certified in 0.5240445137023926s

Edge (269, 142) is False
Certified in 0.5131123065948486s

Edge (269, 101) is False
Certified in 0.5062844753265381s

Edge (269, 173) is False
Certified in 0.583850622177124s

Edge (270, 492) is False
Certified in 0.5055205821990967s

Edge (270, 415) is False
Certified in 0.5385072231292725s

Edge (270, 177) is False
Certified in 0.6231451034545898s

Edge (270, 287) is False
Certified in 0.47002625465393066s

Edge (271, 354) is False
Certified in 0.5590839385986328s

Edge (271, 223) is False
Certified in 0.5523972511291504s

Edge (271, 436) is False
Certified in 0.5625491142272949s

Edge (271, 320) is False
Certified in 0.5917520523071289s



Certified in 0.5895352363586426s

Edge (304, 257) is False
Certified in 0.5977599620819092s

Edge (305, 479) is False
Certified in 0.5855574607849121s

Edge (305, 90) is False
Certified in 0.5795607566833496s

Edge (305, 222) is False
Certified in 0.5673508644104004s

Edge (305, 331) is False
Certified in 0.4740262031555176s

Edge (306, 77) is False
Certified in 0.5780832767486572s

Edge (306, 125) is False
Certified in 0.5605268478393555s

Edge (306, 461) is False
Certified in 0.5512106418609619s

Edge (306, 474) is False
Certified in 0.5619223117828369s

Edge (307, 175) is False
Certified in 0.5301918983459473s

Edge (307, 186) is False
Certified in 0.4826805591583252s

Edge (307, 160) is False
Certified in 0.5186312198638916s

Edge (307, 326) is False
Certified in 0.5400867462158203s

Edge (308, 248) is False
Certified in 0.5442054271697998s

Edge (308, 226) is False
Certified in 0.5445051193237305s

Edge (308, 127) is False
Certified in 0.6249933242797852s

Edge (308, 246) is False

Edge (344, 48) is False
Certified in 0.5468847751617432s

Edge (344, 21) is False
Certified in 0.5176248550415039s

Edge (345, 19) is False
Certified in 0.5349540710449219s

Edge (345, 212) is False
Certified in 0.6017858982086182s

Edge (345, 267) is False
Certified in 0.5022146701812744s

Edge (345, 178) is False
Certified in 0.5262200832366943s

Edge (346, 456) is False
Certified in 0.4827158451080322s

Edge (346, 124) is False
Certified in 0.5457146167755127s

Edge (346, 154) is False
Certified in 0.5117731094360352s

Edge (346, 205) is False
Certified in 0.494365930557251s

Edge (347, 128) is False
Certified in 0.479534387588501s

Edge (347, 5) is False
Certified in 0.47161030769348145s

Edge (347, 381) is False
Certified in 0.5281879901885986s

Edge (347, 192) is False
Certified in 0.4923219680786133s

Edge (348, 187) is False
Certified in 0.5398006439208984s

Edge (348, 210) is False
Certified in 0.4897744655609131s

Edge (348, 27) is False
Certified in 0.47159886360168457s

Edg

KeyboardInterrupt: 